In [24]:
import pandas as pd
import numpy as np
import datetime
import socket
import struct
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'


In [25]:
headerString = "date time time-taken c-ip cs-username cs-auth-group x-exception-id sc-filter-result cs-categories cs(Referer) sc-status s-action cs-method rs(Content-Type) cs-uri-scheme cs-host cs-uri-port cs-uri-path cs-uri-query cs-uri-extension cs(User-Agent) s-ip sc-bytes cs-bytes x-virus-id"
header=headerString.split(" ")
a = pd.read_csv('./Data/message.txt', delimiter="\s+", index_col=False, encoding="utf-8", comment='#',names=header)


C:\Users\VRUma\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (10,16,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [26]:
#drop columns : cs-unsername cs-auth-group  (date prob)
df1 = a.sample(frac=0.5,random_state=200) 
df2 =df1.sample(frac=0.5,random_state=200)

df1=df1.drop(columns=['cs-username','cs-auth-group','date',
                      'time-taken','cs-method',
                      'rs(Content-Type)','cs-uri-path',
                      'cs-uri-query','cs-uri-extension','cs(User-Agent)','s-ip','x-virus-id'])

df3=df1.sample(frac=0.4,random_state=2000)
df3=df3.reset_index(drop=True)


In [27]:
y=df1.tail(1)

def preprocessing(data):
    data=data.dropna()
    data.loc[data['x-exception-id'] == '-', 'x-exception-id'] = ""

    return data

df3

,time,c-ip,x-exception-id,sc-filter-result,cs-categories,cs(Referer),sc-status,s-action,cs-uri-scheme,cs-host,cs-uri-port,sc-bytes,cs-bytes
0,21:17:48,922e695d8efdd9c2,-,OBSERVED,unavailable,http://porn984.xvideos.com/videos/flv/a/b/0/,400,TCP_NC_MISS,http,porn984.xvideos.com,80,221,680
1,20:43:03,b1667d277a0eea24,-,OBSERVED,unavailable,http://thumbs.deviantclip.com/static/player/fl...,403,TCP_NC_MISS,http,mec.deviantclip.com,80,504,920
2,20:48:19,6d98469a3f1de6f4,-,OBSERVED,unavailable,http://video.xnxx.com/tags/public,200,TCP_MISS,http,img100.xvideos.com,80,8402,427
3,21:23:00,dd60f0927674ae4a,-,OBSERVED,unavailable,http://www.01-sy.com/administration/ProductDet...,200,TCP_NC_MISS,http,www.01-sy.com,80,914,1184
4,20:35:51,ee73ceddd52050f0,-,OBSERVED,unavailable,http://www.xhamster.com/movies/101714/swaedyas...,200,TCP_HIT,http,static.xhamster.com,80,1115,887
...,...,...,...,...,...,...,...,...,...,...,...,...,...
208361,20:57:13,2ced9adaec5b4b7d,internal_error,DENIED,unavailable;unavailable,-,0,TCP_ERR_MISS,-,-,0,0,50
208362,20:38:16,2056caa323b6174a,-,OBSERVED,unavailable,http://porn322.xvideos.com/videos/flv/7/f/5/,400,TCP_NC_MISS,http,porn322.xvideos.com,80,221,411
208363,20:54:04,1b28b4270a6cacfd,-,OBSERVED,unavailable,http://www.gamezhero.com/get-game-code/f766406...,200,TCP_NC_MISS,http,www.gamezhero.com,80,3600,1339
208364,20:51:32,d23491d9a5ae7146,-,OBSERVED,unavailable,-,200,TCP_NC_MISS,http,media2.lsops.net,80,182,192


In [28]:
def compute_similarity(string1,stand_val):
    indexes=np.where(string1==stand_val)
    return indexes[0]


def apply_sim(data,column_name,one_line_df,new_col_name):
    if column_name not in data.columns:
        print("Wrong column name")
        return 0
    else :
        list_indexes=compute_similarity(data[column_name].values,one_line_df[column_name].values[0])
        data[new_col_name]=data.index.isin(list_indexes)
        data[new_col_name]=data[new_col_name].astype(int)
    return data

    
df3

,time,c-ip,x-exception-id,sc-filter-result,cs-categories,cs(Referer),sc-status,s-action,cs-uri-scheme,cs-host,cs-uri-port,sc-bytes,cs-bytes
0,21:17:48,922e695d8efdd9c2,-,OBSERVED,unavailable,http://porn984.xvideos.com/videos/flv/a/b/0/,400,TCP_NC_MISS,http,porn984.xvideos.com,80,221,680
1,20:43:03,b1667d277a0eea24,-,OBSERVED,unavailable,http://thumbs.deviantclip.com/static/player/fl...,403,TCP_NC_MISS,http,mec.deviantclip.com,80,504,920
2,20:48:19,6d98469a3f1de6f4,-,OBSERVED,unavailable,http://video.xnxx.com/tags/public,200,TCP_MISS,http,img100.xvideos.com,80,8402,427
3,21:23:00,dd60f0927674ae4a,-,OBSERVED,unavailable,http://www.01-sy.com/administration/ProductDet...,200,TCP_NC_MISS,http,www.01-sy.com,80,914,1184
4,20:35:51,ee73ceddd52050f0,-,OBSERVED,unavailable,http://www.xhamster.com/movies/101714/swaedyas...,200,TCP_HIT,http,static.xhamster.com,80,1115,887
...,...,...,...,...,...,...,...,...,...,...,...,...,...
208361,20:57:13,2ced9adaec5b4b7d,internal_error,DENIED,unavailable;unavailable,-,0,TCP_ERR_MISS,-,-,0,0,50
208362,20:38:16,2056caa323b6174a,-,OBSERVED,unavailable,http://porn322.xvideos.com/videos/flv/7/f/5/,400,TCP_NC_MISS,http,porn322.xvideos.com,80,221,411
208363,20:54:04,1b28b4270a6cacfd,-,OBSERVED,unavailable,http://www.gamezhero.com/get-game-code/f766406...,200,TCP_NC_MISS,http,www.gamezhero.com,80,3600,1339
208364,20:51:32,d23491d9a5ae7146,-,OBSERVED,unavailable,-,200,TCP_NC_MISS,http,media2.lsops.net,80,182,192


In [29]:
def compute_all_sims(data,y):
    all_cols=['x-exception-id','sc-filter-result','cs-categories',
 'sc-status','s-action','cs-uri-scheme','cs-uri-port']
    all_cols
    for col in all_cols:
        apply_sim(data,col,y,'sim')
        if 'final_sim' not in data.columns :
            data['final_sim'] = data['sim']
        else :
            data['final_sim']= data['final_sim'] + data['sim']
    return data
df3=preprocessing(df3)
compute_all_sims(df3,y)

,time,c-ip,x-exception-id,sc-filter-result,cs-categories,cs(Referer),sc-status,s-action,cs-uri-scheme,cs-host,cs-uri-port,sc-bytes,cs-bytes,sim,final_sim
0,21:17:48,922e695d8efdd9c2,,OBSERVED,unavailable,http://porn984.xvideos.com/videos/flv/a/b/0/,400,TCP_NC_MISS,http,porn984.xvideos.com,80,221,680,1,4
1,20:43:03,b1667d277a0eea24,,OBSERVED,unavailable,http://thumbs.deviantclip.com/static/player/fl...,403,TCP_NC_MISS,http,mec.deviantclip.com,80,504,920,1,4
2,20:48:19,6d98469a3f1de6f4,,OBSERVED,unavailable,http://video.xnxx.com/tags/public,200,TCP_MISS,http,img100.xvideos.com,80,8402,427,1,5
3,21:23:00,dd60f0927674ae4a,,OBSERVED,unavailable,http://www.01-sy.com/administration/ProductDet...,200,TCP_NC_MISS,http,www.01-sy.com,80,914,1184,1,5
4,20:35:51,ee73ceddd52050f0,,OBSERVED,unavailable,http://www.xhamster.com/movies/101714/swaedyas...,200,TCP_HIT,http,static.xhamster.com,80,1115,887,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208361,20:57:13,2ced9adaec5b4b7d,internal_error,DENIED,unavailable;unavailable,-,0,TCP_ERR_MISS,-,-,0,0,50,0,0
208362,20:38:16,2056caa323b6174a,,OBSERVED,unavailable,http://porn322.xvideos.com/videos/flv/7/f/5/,400,TCP_NC_MISS,http,porn322.xvideos.com,80,221,411,0,0
208363,20:54:04,1b28b4270a6cacfd,,OBSERVED,unavailable,http://www.gamezhero.com/get-game-code/f766406...,200,TCP_NC_MISS,http,www.gamezhero.com,80,3600,1339,0,0
208364,20:51:32,d23491d9a5ae7146,,OBSERVED,unavailable,-,200,TCP_NC_MISS,http,media2.lsops.net,80,182,192,0,0


In [34]:
def add_urlsize(data):
    data["url_size"] = data["cs(Referer)"].apply(lambda x: len(str(x)))
    return data

def add_hostfreq(data):
    cs_host_frequency = data.groupby(['cs-host']).count().sort_values(by=["url_size"],ascending=False)
    most_frequent = cs_host_frequency.index.tolist()
    quantity = cs_host_frequency['s-action'].tolist()
    frequent_host_dict = dict(zip(most_frequent, quantity))
    data["cs-host-frequency"] = data["cs-host"].apply(lambda x: frequent_host_dict.get(x))
    return data




df3=add_urlsize(df3)
df3=add_hostfreq(df3)
df3

,time,c-ip,x-exception-id,sc-filter-result,cs-categories,cs(Referer),sc-status,s-action,cs-uri-scheme,cs-host,cs-uri-port,sc-bytes,cs-bytes,sim,final_sim,url_size,cs-host-frequency
204030,21:06:45,8d850635c898bc58,,OBSERVED,unavailable,http://googleads.g.doubleclick.net/pagead/ads?...,200,TCP_NC_MISS,http,dispatch.lite.adlesse.com,80,3021,2874,1,5,1989,19
113745,20:48:34,6d98469a3f1de6f4,,OBSERVED,unavailable,http://www.carteltraffic.com/css/style.css?rte...,200,TCP_NC_MISS,http,i.nuseek.com,80,616,2246,1,6,1862,1
198641,21:04:02,6bbcd7ab6dc9b89b,,OBSERVED,unavailable,http://iframe.ivillage.com/iframe_render?k=9eb...,200,TCP_NC_MISS,http,adserving.cpxinteractive.com,80,3040,2152,1,5,1581,2699
105805,21:12:27,0e8fe56c260eb807,,OBSERVED,unavailable,http://www.google.com/uds/afs?q=http%3A%2F%2Fw...,200,TCP_REFRESH_MISS,http,www.google.com,80,651,1969,1,4,1467,468
4192,21:11:46,d4418edeab568d3a,,OBSERVED,unavailable,http://www.facebook.com/extern/login_status.ph...,304,TCP_HIT,http,igor.facemoods.com,80,301,3003,0,3,1456,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92439,20:39:43,bab4984b491b5d31,,OBSERVED,unavailable,-,0,TCP_ERR_MISS,http,down.tutuai.com,80,0,88,0,3,1,25
92421,21:08:41,f794f660fed46811,,OBSERVED,unavailable,-,302,TCP_NC_MISS,http,fxfeeds.mozilla.com,80,617,452,1,5,1,1971
90273,21:05:40,87e72c92d41caf44,,OBSERVED,unavailable,-,200,TCP_MISS,http,v20.lscache2.googlevideo.com,80,12559,504,0,0,1,2
90274,20:47:33,55575920249a09bf,internal_error,DENIED,unavailable;unavailable,-,0,TCP_ERR_MISS,-,-,0,0,54,1,5,1,9477


In [ ]:
def normalization(data):
    df4=data.copy()
    df4['final_sim']=(df4['final_sim']-df4['final_sim'].mean())/df4['final_sim'].std()
    #df4 = df4.drop([283551], axis=0)
    df4['sc-bytes'] = pd.to_numeric(df4['sc-bytes'])
    df4['cs-bytes'] = pd.to_numeric(df4['cs-bytes'])
    df4['sc-bytes']=(df4['sc-bytes']-df4['sc-bytes'].mean())/df4['sc-bytes'].std()
    df4['cs-bytes']=(df4['cs-bytes']-df4['cs-bytes'].mean())/df4['cs-bytes'].std()
    return df4


def normalization2(data):
    df4=data.copy()
    df4['final_sim']=(df4['final_sim']-df4['final_sim'].min())/(df4['final_sim'].max() -df4['final_sim'].min()) 
    #df4 = df4.drop([283551], axis=0)
    df4['sc-bytes'] = pd.to_numeric(df4['sc-bytes'])
    df4['cs-bytes'] = pd.to_numeric(df4['cs-bytes'])
    df4['sc-bytes']=(df4['sc-bytes']-df4['sc-bytes'].min())/(df4['sc-bytes'].max() -df4['sc-bytes'].min()) 
    df4['cs-bytes']=(df4['cs-bytes']-df4['cs-bytes'].min())/(df4['cs-bytes'].max() -df4['cs-bytes'].min()) 
    return df4


dfn=normalization(df3)


In [ ]:
dfn.sort_values(by=['cs-bytes'],ascending=False)
dfn1=dfn.sample(frac=0.1,random_state=1337)

In [ ]:
from sklearn.cluster import DBSCAN

#dfn=dfn.dropna()
clusters = DBSCAN(eps = 0.5, min_samples=100)
clusters.fit(dfn1[['sc-bytes','cs-bytes','final_sim']])
dfn1['labels'] = [str(e) for e in clusters.labels_]

dfn1



In [ ]:
dfn1.groupby('labels').count()

In [ ]:
dfn1.loc[dfn1['labels']=="-1"]

In [ ]:

dfn2=dfn1.loc[dfn1['cs-bytes']<1]
plt.scatter(dfn2['final_sim'],dfn2['cs-bytes'])

In [ ]:
dfn2=dfn2.loc[dfn2['sc-bytes']<1]

plt.scatter(dfn2['final_sim'],dfn2['sc-bytes'])